# nbmodular

> Convert notebooks to modular code.

Convert data science notebooks with poor modularity to fully modular notebooks that are automatically exported as python modules.

## Motivation

In data science, it is usual to develop experimentally and quickly based on notebooks, with little regard to software engineering practices and modularity. It can become challenging to start working on someone else's notebooks with no modularity in terms of separate functions, and a great degree of duplicated code between the different notebooks. This makes it difficult to understand the logic in terms of semantically separate units, see what are the commonalities and differences between the notebooks, and be able to extend, generalize, and configure the current solution.

## Objectives

`nbmodular` is a library conceived with the objective of helping converting the cells of a notebook into separate functions with clear dependencies in terms of inputs and outputs. This is done though a combination of tools which semi-automatically understand the data-flow in the code, based on mild assumptions about its structure. It also helps test the current logic and compare it against a modularized solution, to make sure that the refactored code is equivalent to the original one. 

## Features

- Convert cells to functions.
- The logic of a single function can be written across multiple cells.
- Optional: processed cells can continue to operate as cells or be only used as functions from the moment they are converted.
- Create an additional pipeline function that provides the data-flow from the first to the last function call in the notebook.
- Write all the notebook functions to a separate python module.
- Compare the result of the pipeline with the result of running the original notebook.
- Converted functions act as nodes in a dependency graph. These nodes can optionally hold the values of local variables for inspection outside of the function. This is similar to having a single global scope, which is the original situation. Since this is memory-consuming, it is optional and may not be the default.
- Optional: Once we are able to construct a graph, we may be able to draw it or show it in text, and pass it to ADG processors that can run functions sequentially or in parallel.
- Persist the inputs and outputs of functions, so that we may decide to reuse previous results without running the whole notebook. 
- Optional: if we have the dependency graph and persisted inputs / outputs, we may decide to only run those cells that are predecessors of the current one, i.e., the ones that provide the inputs needed by the current cell. 
- Optional: if we associate a hash code to input data, we may only run the cells when the input data changes. Similarly, if we associate a hash code with AST-converted function code, we may only run those cells whose code has been updated. 
- Optional: have a mechanism for indicating test examples that go into different test python files. 
= Optional: the output of a test cell can be used for assertions, where we require that the current output is the same as the original one.

## Roadmap

- [ ] Convert cell code into functions:
    - [x] Inputs are those variables detected in current cell and also detected in previous cells. This solution requires that created variables have unique names across the notebook. However, even if a new variable with the same name is defined inside the cell, the resulting function is still correct.
    - Outputs are, at this moment, all the variables detected in current cell that are also detected in posterior cells. 
    
- Filter out outputs:
    - Variables detected in current cell, and also detected in previous cells, might not be needed as outputs of the current cell, if the current cell doesn't modify those variables. To detect potential modifications:
        - AST: 
            - If variable appears only on the right of assign statements or in if statements.
            - If it appears only as argument of functions which we know don't modify the variable, such as `print`.
        - Comparing variable values before and after cell:
            - Good for small variables where doing a deep copy is not computationally expensive.
        - Using type checker:
            - Making the variable `Final` and using mypy or other type checker to see if it is modified in the code.
    - Provide hints:
        - Variables that come from other cells might not be needed as output. The remaining are most probably needed.
        - Variables that are modified are clearly needed.

## Install

```sh
pip install nbmodular
```

## Usage

Load ipython extension

In [ ]:
%load_ext nbmodular.core.cell2func

The nbmodular.core.cell2func extension is already loaded. To reload it, use:
  %reload_ext nbmodular.core.cell2func


This allows us to use the following of magic commands, among others


- function <name_of_function_to_define>
- print <name_of_previous_function>
- function_info <name_of_previous_function>
- print_pipeline

Let's go one by one

### function

Use magic command `function` allows to:

- Run the code in the cell normally, and at the same time detect its input and output dependencies and define a function with this input and output:

In [ ]:
%%function get_initial_values
a = 2
b = 3
c = a+b
print (a+b)

5


The code in the previous cell runs as it normally would, but and at the same time defines a function named `get_initial_values` which we can show with the magic command `print`:

In [ ]:
#%debug

In [ ]:
%print get_initial_values

def get_initial_values(b, a):
    a = 2
    b = 3
    c = a+b
    print (a+b)



This function is defined in the notebook space, so we can invoke it:

In [ ]:
%print all

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d
    return b,c,a

def print_all(c, a, d, b):
    print (a, b, c, d)
    return a,b

def analyze():
    x = [1, 2, 3]
    y = [100, 200, 300]
    z = [u+v for u,v in zip(x,y)]
    product = [u*v for u, v in zip(x,y)]

def get_initial_values(b, a):
    a = 2
    b = 3
    c = a+b
    print (a+b)



The inputs and outputs of the function change dynamically every time we add a new function cell. For example, if we add a new function `get_d`:

In [ ]:
%%function get_d
d = 10

In [ ]:
%print get_d

def get_d():
    d = 10



And then a function `add_all` that depend on the previous two functions:

In [ ]:
%%function add_all
a = a + d
b = b + d
c = c + d

In [ ]:
%print add_all

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d



We can see that the uputs from `get_initial_values` and `get_d` change as needed. We can look at all the functions defined so far by using `print all`:

In [ ]:
%print all

def print_all(c, a, d, b):
    print (a, b, c, d)
    return a,b,c,d

def analyze():
    x = [1, 2, 3]
    y = [100, 200, 300]
    z = [u+v for u,v in zip(x,y)]
    product = [u*v for u, v in zip(x,y)]

def get_initial_values(b, a):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d



Similarly the outputs from the last function `add_all` change after we add a other functions that depend on it:

In [ ]:
%%function print_all
print (a, b, c, d)

12 13 15 10


### print

We can see each of the defined functions with `print my_function`, and list all of them with `print all`

In [ ]:
%print all

def analyze():
    x = [1, 2, 3]
    y = [100, 200, 300]
    z = [u+v for u,v in zip(x,y)]
    product = [u*v for u, v in zip(x,y)]

def get_initial_values(b, a):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d
    return b,c,a

def print_all(c, a, d, b):
    print (a, b, c, d)



### print_pipeline

As we add functions to the notebook, a pipeline function is defined. We can print this pipeline with the magic `print_pipeline`

In [ ]:
%print_pipeline

def index_pipeline ():
    analyze ()
    b, c, a = get_initial_values (b, a)
    d = get_d ()
    b, c, a = add_all (d, b, c, a)
    print_all (c, a, d, b)



This shows the data flow in terms of inputs and outputs

And run it:

In [ ]:
self = %cell_processor

In [ ]:
self.function_list

[FunctionProcessor with name analyze, and fields: dict_keys(['original_code', 'name', 'call', 'tab_size', 'arguments', 'return_values', 'unknown_input', 'unknown_output', 'created_variables', 'loaded_names', 'previous_variables', 'argument_variables', 'read_only_variables', 'posterior_variables', 'idx', 'previous_values', 'current_values', 'all_values', 'all_variables', 'code'])
     Arguments: []
     Output: []
     Locals: dict_keys(['x', 'y', 'z', 'product']),
 FunctionProcessor with name get_initial_values, and fields: dict_keys(['original_code', 'name', 'call', 'tab_size', 'arguments', 'return_values', 'unknown_input', 'unknown_output', 'created_variables', 'loaded_names', 'previous_variables', 'argument_variables', 'read_only_variables', 'posterior_variables', 'idx', 'previous_values', 'current_values', 'all_values', 'all_variables', 'code'])
     Arguments: ['b', 'a']
     Output: ['b', 'c', 'a']
     Locals: dict_keys(['a', 'b', 'c']),
 FunctionProcessor with name get_d, and f

In [ ]:
%print all

def analyze():
    x = [1, 2, 3]
    y = [100, 200, 300]
    z = [u+v for u,v in zip(x,y)]
    product = [u*v for u, v in zip(x,y)]

def get_initial_values(b, a):
    a = 2
    b = 3
    c = a+b
    print (a+b)
    return b,c,a

def get_d():
    d = 10
    return d

def add_all(d, b, c, a):
    a = a + d
    b = b + d
    c = c + d
    return b,c,a

def print_all(c, a, d, b):
    print (a, b, c, d)



In [ ]:
index_pipeline()

UnboundLocalError: local variable 'b' referenced before assignment

### function_info

We can get access to many of the details of each of the defined functions by calling `function_info` on a given function name:

In [ ]:
get_initial_values_info = %function_info get_initial_values

This allows us to see:

- The name and value (at the time of running) of the local variables, arguments and results from the function:

In [ ]:
get_initial_values_info.arguments

In [ ]:
get_initial_values_info.current_values

In [ ]:
get_initial_values_info.return_values

We can also inspect the original code written in the cell...

In [ ]:
print (get_initial_values_info.original_code)

the code of the defined function:

In [ ]:
print (get_initial_values_info.code)

.. and the AST trees:

In [ ]:
print (get_initial_values_info.get_ast (code=get_initial_values_info.original_code))

In [ ]:
print (get_initial_values_info.get_ast (code=get_initial_values_info.code))

Now, we can define another function in a cell that uses variables from the previous function.

### cell_processor

This magic allows us to get access to the CellProcessor class managing the logic for running the above magic commands, which can become handy:

In [ ]:
cell_processor = %cell_processor

## Merging function cells

In order to explore intermediate results, it is convenient to split the code in a function among different cells. This can be done by passing the flag `--merge True`

In [ ]:
%%function analyze
x = [1, 2, 3]
y = [100, 200, 300]
z = [u+v for u,v in zip(x,y)]

In [ ]:
z

In [ ]:
%print analyze

In [ ]:
%%function analyze --merge
product = [u*v for u, v in zip(x,y)]

In [ ]:
%print analyze